In [ ]:
import pandas as pd
import numpy as np
import torch.nn as nn
import cv2
from tqdm.notebook import tqdm
from collections import OrderedDict
import os
import torch 
import pytorch_lightning as pl
from torch.utils.data import Dataset,DataLoader
import glob
from torchvision.models import resnet50,resnet18
from albumentations import Compose,Resize,Normalize,CenterCrop
from albumentations.pytorch import ToTensorV2

In [ ]:
class configs:
    img_res = 512
    num_classes = 5
    device = ["cuda" if torch.cuda.is_available() else "cpu"]
    paths = "../input/pytorch-fgvc8-weigthts/Resnet18_512_Checkpoint-ValLossval_loss0.1270-F1val_f10.8357.ckpt"
    classes = np.array(['complex','frog_eye_leaf_spot','powdery_mildew','rust','scab'])


class_map = np.load("../input/pytorch-fgvc8-weigthts/Classes.npy",allow_pickle=True)

In [ ]:
def average_model(paths):
    weights = np.ones((len(paths),))
    weights = weights/weights.sum()
    for i, p in enumerate(paths):
        m = torch.load(p)['state_dict']
        if i == 0:
            averaged_w = OrderedDict()
            for k in m.keys():
                if 'pos' in k: continue
                # remove pl prefix in state dict
                knew = k.replace('model.', '')
                averaged_w[knew] = weights[i]*m[k]
        else:
            for k in m.keys():
                if 'pos' in k: continue
                knew = k.replace('model.', '')
                averaged_w[knew] = averaged_w[knew] + weights[i]*m[k]
    return averaged_w

In [ ]:
samp_data = pd.read_csv("../input/plant-pathology-2021-fgvc8/sample_submission.csv")

In [ ]:
samp_data

In [ ]:
samp_data.image.values

In [ ]:
class FGVCNet(pl.LightningModule):
    def __init__(self):
        super(FGVCNet,self).__init__()
        self.model = resnet18(pretrained=False)
        num_ftrs = self.model.fc.in_features
        
        self.model.fc = nn.Sequential(
            nn.Linear(num_ftrs,num_ftrs),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(num_ftrs,5)
        )

    def forward(self,x):
        return self.model(x)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(),lr=0.01)

    def training_step(self,batch,batch_idx):
        X,y = batch
        y_hat = self.model(X)
        loss_tr = F.cross_entropy(y_hat,y)
        f1_tr = pl.metrics.functional.f1(y_hat,y,12) 
        self.log("TrainLoss",loss_tr,prog_bar=True)
        self.log("TrainF1",f1_tr,prog_bar=True)
        return loss_tr
    
    def validation_step(self,batch,batch_idx):
        X,y = batch
        y_hat = self.model(X)
        loss_val = F.cross_entropy(y_hat,y)
        f1_val = pl.metrics.functional.f1(y_hat,y,12)
        self.log("val_loss",loss_val,prog_bar=True)
        self.log("val_f1",f1_val,prog_bar=True)
        return loss_val

In [ ]:
def Test_Augs():
    return Compose([
        Resize(configs.img_res,configs.img_res),
        Normalize(),
        ToTensorV2()
    ])


class FGVCTest(Dataset):
    def __init__(self,df,root_dir,transforms=None):
        self.df = df
        self.root = root_dir
        self.transforms = transforms
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        img_name = os.path.join(self.root,self.df.loc[idx,'image'])
        img = cv2.imread(img_name)
        if self.transforms is not None:
            img = self.transforms(image=img)['image']
        return img

In [ ]:
dataset = FGVCTest(df=samp_data,root_dir="../input/plant-pathology-2021-fgvc8/test_images",transforms=Test_Augs())
dataset = DataLoader(dataset,batch_size=1,num_workers=4)

In [ ]:
def Ensemble():
    preds = []
    for p in range(len(configs.paths)):
        model = FGVCNet()
        new_model = model.load_from_checkpoint(configs.paths[p])
        new_model.eval()
        pbar = tqdm(enumerate(dataset),total=len(dataset))
        for i,img in pbar:
            new_model = new_model.to(configs.device[0])
            out = new_model(img.to(configs.device[0]))
            out = nn.functional.softmax(out)
            _,pred = torch.max(out,dim=-1)
            sub_df.loc[i,f"Model-{p}"] = pred.detach().cpu().numpy()[0]
    total = sub_df[f"Model-0"].values+sub_df[f"Model-1"].values
    total = total // len(configs.paths)
    samp_data['labels'] = class_map[total.astype(np.int64)]

In [ ]:
def standalone():
    preds = [] 
    c = []
    model = FGVCNet()
    new_model = model.load_from_checkpoint(configs.paths)
    new_model.eval()
    pbar = tqdm(enumerate(dataset),total=len(dataset))
    for i,img in pbar:
        new_model = new_model.to(configs.device[0])
        img = img.to(configs.device[0])
        out = new_model(img)
        out = nn.functional.sigmoid(out)
        _,pred = torch.max(out,dim=-1)
        preds.append(configs.classes[pred.item()])
    return preds  
    #print(preds)

In [ ]:
p = standalone()

In [ ]:
samp_data['labels'] = p

In [ ]:
samp_data.to_csv("submission.csv",index=False)